# Introduction
In this notebook, we will use Lantern to implement a question answering engine. From a given database of pre-existing questions and their answers, we will be able to fetch answers from completely different questions.

For the question-answer data that will make up our knowledge base, we will be using the [InsuranceQA Corpus](https://github.com/shuzi/insuranceQA).

We make the core functionality very simple using the python library Towhee, so that you can start hacking your own question answering engine.


# Setup Postgres

We install postgres and its dev tools (necessary to build lantern from source). We also start postgres, and set up a user 'postgres' with password 'postgres' and create a database called 'ourdb'




In [ ]:
# We install postgres and its dev tools
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql postgresql-server-dev-all
#  Start postgres
!sudo service postgresql start

# Create user, password, and db
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS ourdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE ourdb;'

W: Failed to fetch https://cloud.r-project.org/bin/linux/ubuntu/jammy-cran40/InRelease  Could not resolve 'cloud.r-project.org'
W: Some index files failed to download. They have been ignored, or old ones used instead.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 26.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package logrotate.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../00-logrotate_3.19.0-1ubuntu1.1_amd64.deb ...
Unpacking logrotate (3.19.0-1ubuntu1.1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../01-netbase_6.3_all.deb

# Install Lantern and build it from source

In [ ]:
!git clone --recursive https://github.com/lanterndata/lantern.git

Cloning into 'lantern'...
remote: Enumerating objects: 2656, done.
remote: Counting objects: 100% (1452/1452), done.
remote: Compressing objects: 100% (505/505), done.
remote: Total 2656 (delta 1124), reused 1041 (delta 941), pack-reused 1204
Receiving objects: 100% (2656/2656), 627.19 KiB | 11.40 MiB/s, done.
Resolving deltas: 100% (1740/1740), done.
Submodule 'third_party/hnswlib' (https://github.com/ngalstyan4/hnswlib) registered for path 'third_party/hnswlib'
Submodule 'third_party/usearch' (https://github.com/ngalstyan4/usearch) registered for path 'third_party/usearch'
Cloning into '/content/lantern/third_party/hnswlib'...
remote: Enumerating objects: 1723, done.        
remote: Counting objects: 100% (333/333), done.        
remote: Compressing objects: 100% (40/40), done.        
remote: Total 1723 (delta 306), reused 293 (delta 293), pack-reused 1390        
Receiving objects: 100% (1723/1723), 530.50 KiB | 11.05 MiB/s, done.
Resolving deltas: 100% (1097/1097), done.
Cloning i

In [ ]:
# We build lantern from source
%cd lantern
!mkdir build
%cd build
!pwd
!cmake ..
!make install

/content/lantern
/content/lantern/build
/content/lantern/build
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Build type: 
-- Found pg_config as /usr/bin/pg_config
-- Found postgres binary at /usr/lib/postgresql/14/bin/postgres
-- P

# Installing other Prerequisites

In [ ]:
!python -m pip install -q towhee towhee.models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.0/222.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.4 MB/s eta 0:00:00


# Gathering Data
Let's download a subset of the [InsuranceQA corpus](https://github.com/shuzi/insuranceQA) we mentioned above. It contains 1000 pairs of questions and answers related to insurance.

The data contains three columns: an `id`, a `question`, and its corresponding `answer`.

In [ ]:
!curl -L https://github.com/towhee-io/examples/releases/download/data/question_answer.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  595k  100  595k    0     0   664k      0 --:--:-- --:--:-- --:--:--  664k


In [ ]:
import pandas as pd

df = pd.read_csv('question_answer.csv')
df.head()

,id,question,answer
0,0,Is Disability Insurance Required By Law?,Not generally. There are five states that requ...
1,1,Can Creditors Take Life Insurance After ...,If the person who passed away was the one with...
2,2,Does Travelers Insurance Have Renters Ins...,One of the insurance carriers I represent is T...
3,3,Can I Drive A New Car Home Without Ins...,Most auto dealers will not let you drive the c...
4,4,Is The Cash Surrender Value Of Life Ins...,Cash surrender value comes only with Whole Lif...


# Create Postgres Table

Now let's set up `psycopg2` with postgres, and enable the lantern extension


In [ ]:
import psycopg2

# We use the dbname, user, and password that we specified above
conn = psycopg2.connect(
    dbname="ourdb",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432" # default port for Postgres
)

# Get a new cursor
cursor = conn.cursor()

# Execute the query to load the Lantern extension in
cursor.execute("CREATE EXTENSION IF NOT EXISTS lantern;")

conn.commit()
cursor.close()

Now let's create the table that we will use to store the data we will reference against. We'll call the table `questions_answers`, and it will have 4 columns. The first 3 correspond to the columns in our dataset we downloaded above: an `id`, the text content of the `question`, and its corresponding `answer`. Lastly, we will store the embedding we compute for each question in the column `vector`. Note that we make `vector` of type real array (`real[]`). We can add a dimension, like `real[768]`, but note that this dimension specified here is just syntactic sugar in postgres, and is not enforced.

In [ ]:
# Create the table
cursor = conn.cursor()

TABLE_NAME = "questions_answers"
create_table_query = f"CREATE TABLE {TABLE_NAME} (id integer, question text, answer text, vector real[]);"

cursor.execute(create_table_query)

conn.commit()
cursor.close()

# Computing Embeddings and Inserting Data

Let's compute the embeddings of the questions in our dataset and insert them into our database. To do this, we will use Facebook's `dpr-ctx_encoder-single-nq-base` model that is included in Towhee. Note that this model creates vector of size 768, and so this is the dimensionality that we will specify later.

Towhee provides a method-chaining style API that we will use to create a pipeline to allow us to compute the embedding, and insert it into our database-- all in one pipeline.

Note that we include a normalization step in the pipeline, which is done so that we can use the `L2-squared` metric in our index later, when running vector search.

Also, note that the majority of the time here is spent on computing the embedding using the aforementioned model.


In [ ]:
%%time
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection

# Define the processing pipeline
def insert_row(id, vec, question, answer):
    vector = [float(x) for x in vec]
    cursor.execute(f"INSERT INTO {TABLE_NAME} (id, question, answer, vector) VALUES (%s, %s, %s, %s);", (id, question, answer, vector))
    return True

insert_pipe = (
    pipe.input('id', 'question', 'answer')
        .map('question', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        # We normalize the embedding here
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map(('id', 'vec', 'question', 'answer'), 'insert_status', insert_row)
        .output()
)

# Insert data
import csv
cursor = conn.cursor()

with open('question_answer.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        insert_pipe(*row)

conn.commit()
cursor.close()

CPU times: user 2min 25s, sys: 3.25 s, total: 2min 28s
Wall time: 2min 57s


# Creating an Index
Now that we have inserted the embeddings into our database, we need to construct an index in postgres using lantern. This is important because the index will tell allow postgres to use lantern when performing vector search.

Note that we specify L2-squared (squared Euclidean distance) as the distance metric, as we mentioned earlier. Also, as a good practice, we specify the dimension of the index (although lantern can infer it from the vector's we've already inserted).

In [ ]:
cursor = conn.cursor()

cursor.execute(f"CREATE INDEX ON {TABLE_NAME} USING hnsw (vector dist_l2sq_ops) WITH (dim=768);")

conn.commit()
cursor.close()

# Performing Similarity Search

Now that we have embedded our questions, let's implement the bulk of the question-answering engine: the vector search!

The idea here is that we will start with a query question. You can imagine that this is a question that a user has submitted to our engine. Our goal is to find an answer to this question.

We will do this by embedding the query question. Then, we will perform a vector search to find the most semantically similar question in our database. Then, we will retrieve the answer to this most similar question from our database, and we will serve that as the answer to the query question.

Let's see this in action by specifying the pipeline we will use, and an example of this process with `QUERY_QUESTION` below:

In [ ]:
conn.rollback()
cursor = conn.cursor()

# We only need to set this at the beginning of a session
cursor.execute("SET enable_seqscan = false;")
conn.commit()

def vector_search(vec):
  query_vector = str([float(x) for x in vec])
  cursor.execute(f"SELECT question AS similar_question, answer FROM {TABLE_NAME} ORDER BY vector <-> ARRAY{query_vector} LIMIT 1;")
  record = cursor.fetchall()[0]
  return record

ans_pipe = (
    pipe.input('question')
        .map('question', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', ('similar_question','answer'), vector_search)
        .output('question', 'similar_question', 'answer')
)

QUERY_QUESTION = "How much does disability insurance cost?"

ans = ans_pipe(QUERY_QUESTION)
ans = DataCollection(ans)
ans.show()

cursor.close()

question,similar_question,answer
How much does disability insurance cost?,How Much Is Disability Insurance On Average?,"On average, long term disability insurance costs 1% to 3% of the annual salary for men; slightly higher for women. There are man..."


# Conclusion
As we can see, we are able to obtain a very similar question to the original query question, and we retrieve the answer we stored alongside this similar question to answer the original question.

And that's how you can implement a simple Question Answering engine using Lantern! There are many approaches to how we go from the query question to a certain row in our database, but the one outlined above is a very straightforward, simple, and efficient approach. The premise behind all these approaches remains the same, however: use vector search to make the connection between the user question and our database which holds our unstructured knowledge-base data.




### Cleanup

In [ ]:
# Close the postgres connection
conn.close()